In [1]:
from __future__ import annotations

import json
import cProfile
import pstats

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex,
    DB
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
data_source = DataSource(
    rec=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [5]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [7]:
with cProfile.Profile() as pr:
  db = DB()
  rec = await data_source.load(data, db=db)
  
  stats = pstats.Stats(pr)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lworm\AppData\Local\Temp\ipykernel_10504\3510191985.py:3 in <module>                    │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│   2   db = DB()                                                                                  │
│ ❱ 3   rec = await data_source.load(data, db=db)                                                  │
│   4                                                                                              │
│   5   stats = pstats.Stats(pr)                                                                   │
│   6                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:580 in load                   │
│                                                                                                  │
│   577 │   │   Returns:                                                                           │
│   578 │   │   │   A Record instance                                                              │
│   579 │   │   """                                                                                │
│ ❱ 580 │   │   return await _load_record(self.rec, self, data, self._obj_cache, db)               │
│   581                                                                                            │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:477 in _load_record           │
│                                                                                                  │
│   474 │   }                                                                                      │
│   475 │                                                                                          │
│   476 │   rec_dict: dict[Col | RelSet, Any] = {a[0]: a[2] for a in attrs.values()}               │
│ ❱ 477 │   rec = rec_type._from_partial_dict(rec_dict, name_keys=False)                           │
│   478 │                                                                                          │
│   479 │   if rec._index in obj_cache[rec_type]:                                                  │
│   480 │   │   return obj_cache[rec_type][rec._index]                                             │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:1513 in _from_partial_dict           │
│                                                                                                  │
│   1510 │   │   │   p.name: data.get(p if not name_keys else p.name, State.undef)  # type: ignor  │
│   1511 │   │   │   for p in cls._props.values()                                                  │
│   1512 │   │   }                                                                                 │
│ ❱ 1513 │   │   return cls(**args)                                                                │
│   1514 │                                                                                         │
│   1515 │   def __repr__(self) -> str:  # noqa: D105                                              │
│   1516 │   │   return self._to_dict(name_keys=True).__repr__()                                   │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:1240 in __init__                     │
│                                                            

In [ ]:
rec

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")